In [15]:
import pandas as pd

df = pd.read_csv("headings.csv")

In [16]:
df.shape

(484, 6)

In [17]:
df

,Heading,up,down,link,text,date
0,Пишем трассировщик лучей для ZX Spectrum ...,86.0,1.0,https://habr.com/ru/articles/788984/ ...,Я люблю трассировщики лучей и даже посвятил им...,2024-01-26
1,Что находится внутри физического Архива Интерн...,60.0,0.0,https://habr.com/ru/companies/beeline_tech/art...,\n ...,2024-01-26
2,Мануальные объективы на цифровых камерах: хоро...,40.0,2.0,https://habr.com/ru/companies/ruvds/articles/7...,\n ...,2024-01-26
3,"Рыцарь-демон, Intel Xeon Phi в 2024-м, часть 0...",36.0,0.0,https://habr.com/ru/articles/788790/ ...,Изображение НЕ сгенерировано нейросетьюВы точн...,2024-01-26
4,Ветер и кольца: об экзотических звёздных класс...,37.0,3.0,https://habr.com/ru/articles/788986/ ...,Большинство звёзд относится к основным спектра...,2024-01-26
...,...,...,...,...,...,...
479,Учимся понимать события подсистемы аудита Linu...,1.0,0.0,https://habr.com/ru/companies/pt/articles/7890...,Приветствую всех любителей изучать новое. Меня...,2024-01-29
480,"Leo, ИИ-помощник браузера Brave, теперь доступ...",1.0,0.0,https://habr.com/ru/companies/brave/articles/7...,"Три месяца назад мы открыли доступ к Leo, ИИ-п...",2024-01-29
481,Представление родственных связей (генеалогичес...,1.0,0.0,https://habr.com/ru/articles/783938/ ...,Прочитал статью Фамильный вики-движок Bonsai: ...,2024-01-29
482,Мобильная разработка за неделю #523 (22 — 28 я...,2.0,1.0,https://habr.com/ru/companies/productivity_ins...,В новом дайджесте открытие сторонних магазинов...,2024-01-29
